In [84]:
import pandas as pd
import numpy as np
import uuid
import os
import jsonlines
import datetime
import uuid

In [85]:
all_sessions_raw = pd.read_json("../output/raw/sessions_all_crawlies.json")

In [86]:
all_sessions_raw.iloc[0]

id                       3daf5a49-1466-4b98-b6d0-1cb5391e5bd1
class                                                 session
range                                                     1.0
order                                                     2.0
event_id                 a12c53cf-a1d6-4def-9c54-a765ccdc3de8
title            Lymphatics, lipoproteins and the endothelium
location                                               Hall C
start_time                                2021-06-01 14:30:00
end_time                                  2021-06-01 15:10:00
content_id                                                NaN
role                                                      NaN
full_name                                                 NaN
first_name                                                NaN
last_name                                                 NaN
second_name                                               NaN
country                                                   NaN
session_

## All sessions

In [87]:
# filter sessions from all session data (sessions, pres, persons)
sessionsdf = all_sessions_raw[all_sessions_raw["class"] == "session"][[
        "id", "range", "order", "event_id", "title", "location", "start_time", "end_time"
    ]]

In [88]:
sessionsdf.head()

,id,range,order,event_id,title,location,start_time,end_time
0,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,1.0,2.0,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,"Lymphatics, lipoproteins and the endothelium",Hall C,2021-06-01 14:30:00,2021-06-01 15:10:00
11,cb55033f-259d-4b2a-bfee-0207fd941a49,1.0,1.0,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Liver as the center of lipid metabolism,Hall E,2021-06-01 14:30:00,2021-06-01 16:00:00
22,3b09e701-446d-41f2-a06a-04a9f38aae6e,1.0,8.0,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Diet and CVD,Hall B (Live Q&A),2021-06-01 12:30:00,2021-06-01 14:00:00
40,a47d4734-499c-4d06-9128-2327607c7bbe,1.0,3.0,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Aortic valve stenosis and Lp(a): fro...,Live Streamed,2021-06-01 16:00:00,2021-06-01 17:30:00
61,111e3af4-b089-4049-a890-6a20b78d6223,1.0,5.0,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Cell metabolism and vascular health,Hall D,2021-06-01 14:30:00,2021-06-01 16:00:00


In [89]:
# convert range and order to integers
sessionsdf["range"] = sessionsdf["range"].apply(lambda x: int(x))
sessionsdf["order"] = sessionsdf["order"].apply(lambda x: int(x))

In [90]:
# visualize duplicated sessions
sessionsdf[sessionsdf.duplicated(['title'])]

,id,range,order,event_id,title,location,start_time,end_time


In [91]:
len(sessionsdf)

45

In [92]:
event_id = sessionsdf.iloc[0]["event_id"]

In [93]:
# sessionsdf["start_time"]

In [94]:
sessionsdf.columns

Index(['id', 'range', 'order', 'event_id', 'title', 'location', 'start_time',
       'end_time'],
      dtype='object')

In [95]:
presentationsdf = all_sessions_raw[all_sessions_raw["class"] == "presentation"][[
        "id", "title", "order", "start_time", "end_time", "session_id", "description",
        "session_order", "session_range", "session_type"]]

In [96]:
sessionsdf = sessionsdf.merge(
    presentationsdf[["session_id", "session_type"]], left_on="id", right_on="session_id", how="left").drop_duplicates()

In [97]:
sessionsdf

,id,range,order,event_id,title,location,start_time,end_time,session_id,session_type
0,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,1,2,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,"Lymphatics, lipoproteins and the endothelium",Hall C,2021-06-01 14:30:00,2021-06-01 15:10:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,Advanced Clinical Seminars
4,cb55033f-259d-4b2a-bfee-0207fd941a49,1,1,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Liver as the center of lipid metabolism,Hall E,2021-06-01 14:30:00,2021-06-01 16:00:00,cb55033f-259d-4b2a-bfee-0207fd941a49,Advanced Clinical Seminars
8,3b09e701-446d-41f2-a06a-04a9f38aae6e,1,8,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Diet and CVD,Hall B (Live Q&A),2021-06-01 12:30:00,2021-06-01 14:00:00,3b09e701-446d-41f2-a06a-04a9f38aae6e,Lipoproteins and Metabolism
16,a47d4734-499c-4d06-9128-2327607c7bbe,1,3,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Aortic valve stenosis and Lp(a): fro...,Live Streamed,2021-06-01 16:00:00,2021-06-01 17:30:00,a47d4734-499c-4d06-9128-2327607c7bbe,Genetics
25,111e3af4-b089-4049-a890-6a20b78d6223,1,5,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Cell metabolism and vascular health,Hall D,2021-06-01 14:30:00,2021-06-01 16:00:00,111e3af4-b089-4049-a890-6a20b78d6223,Genetics
32,801b711a-cdb2-4f03-a192-803040604d95,1,6,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Atherosclerosis in ageing societies,Live Streamed,2021-06-02 15:30:00,2021-06-02 16:30:00,801b711a-cdb2-4f03-a192-803040604d95,Advanced Clinical Seminars
36,0580a215-6282-4d05-9bd8-69b4a49001bb,1,4,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Influence of genes on the susceptibi...,Live Streamed,2021-05-31 15:00:00,2021-05-31 16:30:00,0580a215-6282-4d05-9bd8-69b4a49001bb,Genetics
45,d31d3129-d24e-4daa-a7b1-3d6a18cfccd9,1,7,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Lipid metabolism and atherothrombosis,Live Streamed,2021-05-31 11:30:00,2021-05-31 13:00:00,d31d3129-d24e-4daa-a7b1-3d6a18cfccd9,Lipoproteins and Metabolism
54,10ce7b00-8a9c-4195-9e1e-7c5b6b4714fc,1,0,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Current insights into the role of gut microbio...,Hall D,2021-05-31 16:00:00,2021-05-31 17:30:00,10ce7b00-8a9c-4195-9e1e-7c5b6b4714fc,Advanced Clinical Seminars
58,f67357ce-c4e3-4419-a348-6455ccceb932,4,10,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Sponsored Session 5 (not Included in the main ...,Industry session hall,2021-06-02 11:00:00,2021-06-02 11:45:00,f67357ce-c4e3-4419-a348-6455ccceb932,Industry Sponsored Session


In [98]:
sessionsdf[sessionsdf["title"]=="Meet the experts - Overcoming barriers to population health-through siRNA based therapies"]

,id,range,order,event_id,title,location,start_time,end_time,session_id,session_type
258,2132b255-b30c-4d5f-8bcf-b2e448bf3b3b,4,1,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Meet the experts - Overcoming barriers to popu...,Meet the expert session hall,2021-05-31 16:30:00,2021-05-31 17:00:00,2132b255-b30c-4d5f-8bcf-b2e448bf3b3b,Meet the experts


In [99]:
# add non referenced sessions
sessionsdf = sessionsdf.append(pd.DataFrame([{
    "id": str(uuid.uuid4()),
    "range": 4, "order": 1.3, "event_id": event_id, "title": "Meet the experts - Addressing residual risk beyond LDL-c in high or very high-risk patients",
    "location": "Meet the expert session hall",
    "start_time": datetime.datetime.strptime("20210531-1730", '%Y%m%d-%H%M'),
    "end_time": datetime.datetime.strptime("20210531-1800", '%Y%m%d-%H%M')}]))

In [100]:
# add non referenced sessions
sessionsdf = sessionsdf.append(pd.DataFrame([{
    "id": str(uuid.uuid4()),
    "range": 4, "order": 1.6, "event_id": event_id, "title": "Meet the experts - Novel pathways for intensive lipid lowering",
    "location": "Meet the expert session hall",
    "start_time": datetime.datetime.strptime("20210601-1630", '%Y%m%d-%H%M'),
    "end_time": datetime.datetime.strptime("20210601-1700", '%Y%m%d-%H%M')}]))

In [101]:
len(sessionsdf)

47

In [102]:
sessionsdf[-2:]

,id,range,order,event_id,title,location,start_time,end_time,session_id,session_type
0,7d0e7bfe-1f4c-4c71-9541-967d1c54a2d8,4,1.3,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Meet the experts - Addressing residual risk be...,Meet the expert session hall,2021-05-31 17:30:00,2021-05-31 18:00:00,NaN,NaN
0,7152bbd6-026f-4cce-8861-02e6996a2405,4,1.6,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Meet the experts - Novel pathways for intensiv...,Meet the expert session hall,2021-06-01 16:30:00,2021-06-01 17:00:00,NaN,NaN


In [65]:
# Don't forget session chairs

In [103]:
# Reorder sessions
sessionsdf["rangeorder"] = sessionsdf["range"]*100 + sessionsdf["order"]
sessionsdf.sort_values(by="rangeorder", inplace=True)
sessionsdf.reset_index(inplace=True, drop=True)
sessionsdf["order"] = sessionsdf.index + 1

In [104]:
sessionsdf.drop(columns=["range"], inplace=True)

In [105]:
sessionsdf.head()

,id,order,event_id,title,location,start_time,end_time,session_id,session_type,rangeorder
0,10ce7b00-8a9c-4195-9e1e-7c5b6b4714fc,1,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Current insights into the role of gut microbio...,Hall D,2021-05-31 16:00:00,2021-05-31 17:30:00,10ce7b00-8a9c-4195-9e1e-7c5b6b4714fc,Advanced Clinical Seminars,100.0
1,cb55033f-259d-4b2a-bfee-0207fd941a49,2,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Liver as the center of lipid metabolism,Hall E,2021-06-01 14:30:00,2021-06-01 16:00:00,cb55033f-259d-4b2a-bfee-0207fd941a49,Advanced Clinical Seminars,101.0
2,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,3,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,"Lymphatics, lipoproteins and the endothelium",Hall C,2021-06-01 14:30:00,2021-06-01 15:10:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,Advanced Clinical Seminars,102.0
3,a47d4734-499c-4d06-9128-2327607c7bbe,4,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Aortic valve stenosis and Lp(a): fro...,Live Streamed,2021-06-01 16:00:00,2021-06-01 17:30:00,a47d4734-499c-4d06-9128-2327607c7bbe,Genetics,103.0
4,0580a215-6282-4d05-9bd8-69b4a49001bb,5,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Workshop: Influence of genes on the susceptibi...,Live Streamed,2021-05-31 15:00:00,2021-05-31 16:30:00,0580a215-6282-4d05-9bd8-69b4a49001bb,Genetics,104.0


In [106]:
sessionsdf[-2:]

,id,order,event_id,title,location,start_time,end_time,session_id,session_type,rangeorder
45,1843e4ff-49d9-4fa5-a46e-07e6154d6e76,46,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Sponsored Session 4 (not Included in the main ...,Industry session hall,2021-06-01 11:00:00,2021-06-01 11:45:00,1843e4ff-49d9-4fa5-a46e-07e6154d6e76,Industry Sponsored Session,409.0
46,f67357ce-c4e3-4419-a348-6455ccceb932,47,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Sponsored Session 5 (not Included in the main ...,Industry session hall,2021-06-02 11:00:00,2021-06-02 11:45:00,f67357ce-c4e3-4419-a348-6455ccceb932,Industry Sponsored Session,410.0


In [107]:
with jsonlines.open('../output/structured/Session.jsonl', mode='w') as writer:
    sessionsdf["start_time"] = sessionsdf["start_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
    sessionsdf["end_time"] = sessionsdf["end_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
    writer.write_all(sessionsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

## All presentations, including e-posters

In [108]:
# Filter presentations
all_sessions_raw[all_sessions_raw["class"]=="presentation"].iloc[250]

id               51a89650-e7f0-445f-8b5f-dfffb3e499cc
class                                    presentation
range                                             NaN
order                                             1.0
event_id         a12c53cf-a1d6-4def-9c54-a765ccdc3de8
title                              Supported by Amgen
location                                          NaN
start_time                        2021-05-30 09:45:00
end_time                          2021-05-30 09:45:00
content_id                                        NaN
role                                              NaN
full_name                                         NaN
first_name                                        NaN
last_name                                         NaN
second_name                                       NaN
country                                           NaN
session_order                                     5.0
session_range                                     4.0
session_id       a35bbe75-46

In [109]:
presentationsdf = all_sessions_raw[all_sessions_raw["class"] == "presentation"][[
        "id", "title", "order", "start_time", "end_time", "session_id", "description",
        "session_order", "session_range"]]

In [110]:
[item for item in presentationsdf.session_id if item not in list(sessionsdf.id)]

[]

In [111]:
presentationsdf.head()

,id,title,order,start_time,end_time,session_id,description,session_order,session_range
3,28dd32ad-8624-47ae-b47a-90ecbdcc4c9a,Introduction,0.0,2021-06-01 14:30:00,2021-06-01 14:33:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,NaN,2.0,1.0
5,7e23090a-d6b3-4631-8b30-66dc644481e8,Lymphatics in the Regulation of Intestinal Lip...,1.0,2021-06-01 14:33:00,2021-06-01 14:48:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,NaN,2.0,1.0
7,f890f8c7-be4b-484e-bcef-cd08e2c73680,Lymphatic System in Cardiovascular Medicine,2.0,2021-06-01 14:48:00,2021-06-01 15:03:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,NaN,2.0,1.0
9,afddf7fc-1f52-439a-a466-1d5562792ebb,Conclusion,3.0,2021-06-01 15:03:00,2021-06-01 15:06:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,NaN,2.0,1.0
14,b9562485-d68b-4086-8610-a6ad50f27b9b,Introduction,0.0,2021-06-01 14:30:00,2021-06-01 14:33:00,cb55033f-259d-4b2a-bfee-0207fd941a49,NaN,1.0,1.0


In [112]:
presentationsdf["order"] = presentationsdf["order"].apply(lambda x: int(x))
presentationsdf["session_order"] = presentationsdf["session_order"].apply(lambda x: int(x))
presentationsdf["session_range"] = presentationsdf["session_range"].apply(lambda x: int(x))

In [113]:
presentationsdf

,id,title,order,start_time,end_time,session_id,description,session_order,session_range
3,28dd32ad-8624-47ae-b47a-90ecbdcc4c9a,Introduction,0,2021-06-01 14:30:00,2021-06-01 14:33:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,NaN,2,1
5,7e23090a-d6b3-4631-8b30-66dc644481e8,Lymphatics in the Regulation of Intestinal Lip...,1,2021-06-01 14:33:00,2021-06-01 14:48:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,NaN,2,1
7,f890f8c7-be4b-484e-bcef-cd08e2c73680,Lymphatic System in Cardiovascular Medicine,2,2021-06-01 14:48:00,2021-06-01 15:03:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,NaN,2,1
9,afddf7fc-1f52-439a-a466-1d5562792ebb,Conclusion,3,2021-06-01 15:03:00,2021-06-01 15:06:00,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,NaN,2,1
14,b9562485-d68b-4086-8610-a6ad50f27b9b,Introduction,0,2021-06-01 14:30:00,2021-06-01 14:33:00,cb55033f-259d-4b2a-bfee-0207fd941a49,NaN,1,1
...,...,...,...,...,...,...,...,...,...
715,7eb4fa0a-3da0-41c7-93df-587f5e313591,O003 - Identification of Novel Lipid Droplet F...,2,2021-05-31 11:14:00,2021-05-31 11:21:00,7b7699e4-8a54-41ff-b55c-7f395bd096ea,Background and Aims Macrophage autophagy is a ...,0,4
717,3f165b28-9ba4-4a89-bb6a-6dd55df50ad1,O004 - Role of the Interleukin-1 Receptor/Toll...,3,2021-05-31 11:21:00,2021-05-31 11:28:00,7b7699e4-8a54-41ff-b55c-7f395bd096ea,Background and Aims TIR8/SIGIRR dampens the ex...,0,4
719,77dc56d1-ed3a-4c5d-995b-54faa51b770b,O005 - MLb-LDLr: A Machine Learning Model for ...,4,2021-05-31 11:28:00,2021-05-31 11:35:00,7b7699e4-8a54-41ff-b55c-7f395bd096ea,Background and Aims Cardiovascular disease (CV...,0,4
721,014437d6-ba9a-4f2c-a1d6-b36a26355fed,O006 - Tissue-nonspecific alkaline phosphatase...,5,2021-05-31 11:35:00,2021-05-31 11:42:00,7b7699e4-8a54-41ff-b55c-7f395bd096ea,Background and Aims The calcium score predicts...,0,4


In [114]:
presentationsdf["rangeorder"] = presentationsdf[
    "session_range"]*10000 + presentationsdf["session_order"]*100 + presentationsdf["order"]
presentationsdf["rangeorder"] = presentationsdf["rangeorder"].apply(int)
presentationsdf.drop(columns=["session_range", "session_order", "order"], inplace=True)

In [115]:
presentationsdf.sort_values(by="rangeorder", inplace=True)

In [116]:
presentationsdf.reset_index(inplace=True, drop=True)
presentationsdf["order"] = presentationsdf.index + 1
presentationsdf.drop(columns=["rangeorder"], inplace=True)

In [117]:
presentationsdf["description"] = presentationsdf["description"].apply(
    lambda x: np.nan if pd.isnull(x) else x)

In [118]:
presentationsdf.head()

,id,title,start_time,end_time,session_id,description,order
0,b5156519-1dbd-4f93-9571-28da0ee49d9e,Introduction,2021-05-31 16:00:00,2021-05-31 16:03:00,10ce7b00-8a9c-4195-9e1e-7c5b6b4714fc,NaN,1
1,3727431e-83b8-4199-b332-ee90252f70e5,Is gut microbiota a risk factor for cardiovasc...,2021-05-31 16:03:00,2021-05-31 16:18:00,10ce7b00-8a9c-4195-9e1e-7c5b6b4714fc,NaN,2
2,069b8a9d-4032-487c-a104-eb03cfc97650,The Impact of Diet on Gut Microbiota in Health...,2021-05-31 16:18:00,2021-05-31 16:33:00,10ce7b00-8a9c-4195-9e1e-7c5b6b4714fc,NaN,3
3,7553bfa9-cf7a-4796-81f9-50a573cb05e5,Conclusion,2021-05-31 16:33:00,2021-05-31 16:36:00,10ce7b00-8a9c-4195-9e1e-7c5b6b4714fc,NaN,4
4,b9562485-d68b-4086-8610-a6ad50f27b9b,Introduction,2021-06-01 14:30:00,2021-06-01 14:33:00,cb55033f-259d-4b2a-bfee-0207fd941a49,NaN,5


In [119]:
sessionsdf["event_id"].unique()

array(['a12c53cf-a1d6-4def-9c54-a765ccdc3de8'], dtype=object)

In [120]:
[item for item in presentationsdf.session_id if item not in list(sessionsdf.id)]

[]

In [121]:
with jsonlines.open('../output/structured/Presentation.jsonl', mode='w') as writer:
    presentationsdf["start_time"] = presentationsdf["start_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
    presentationsdf["end_time"] = presentationsdf["end_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
    writer.write_all(presentationsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

## Persons

In [122]:
all_sessions_raw["class"].unique()

array(['session', 'session_person', 'presentation', 'presentation_person'],
      dtype=object)

In [123]:
personsdf = all_sessions_raw[all_sessions_raw["class"].isin(['session_person', 'presentation_person'])][[
    "event_id", "full_name", "first_name", "last_name", "second_name", "country", "title"
]]

In [124]:
personsdf.head()

,event_id,full_name,first_name,last_name,second_name,country,title
1,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Kari Alitalo,Kari,Alitalo,,Finland,
2,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Giuseppina Caligiuri,Giuseppina,Caligiuri,,France,
4,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Giuseppina Caligiuri,Giuseppina,Caligiuri,,France,
6,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Gary Lewis,Gary,Lewis,,Canada,
8,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Kari Alitalo,Kari,Alitalo,,Finland,


In [125]:
# add non referenced sessions
personsdf = personsdf.append(pd.DataFrame([{
    "event_id": event_id, "fullname": "Kausik K. Ray",
    "first_name": "Kausik",
    "last_name": "Ray",
    "second_name": "K.",
    "country": "United Kingdom",
    "title": ""}]))

In [126]:
personsdf = personsdf.append(pd.DataFrame([{
    "event_id": event_id, "fullname": "Alberico L. Catapano",
    "first_name": "Alberico",
    "last_name": "Catapano",
    "second_name": "L.",
    "country": "Italy",
    "title": ""}]))

In [127]:
len(personsdf)

415

In [128]:
personsdf = personsdf.drop_duplicates(subset="full_name", keep='first')

In [129]:
len(personsdf)

194

In [130]:
personsdf["id"] = [str(uuid.uuid4()) for i in range(len(personsdf))]

In [131]:
personsdf.head()

,event_id,full_name,first_name,last_name,second_name,country,title,fullname,id
1,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Kari Alitalo,Kari,Alitalo,,Finland,,NaN,d3f946fd-0513-4970-8761-efaa6450e40c
2,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Giuseppina Caligiuri,Giuseppina,Caligiuri,,France,,NaN,6b87fb58-3335-4ec7-9e44-07724aa2666b
6,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Gary Lewis,Gary,Lewis,,Canada,,NaN,190ceddd-414c-4db5-b2dd-2b95c265c723
12,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Vesa M. Olkkonen,Vesa,Olkkonen,M.,Finland,,NaN,b68458e1-6ad3-4345-bdb9-ffb1f36f7db6
13,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Stefano Romeo,Stefano,Romeo,,Sweden,,NaN,827ad9a3-8e1e-4bb8-b013-9c9fec17581f


In [132]:
personsdf["event_id"].unique()[0]

'a12c53cf-a1d6-4def-9c54-a765ccdc3de8'

In [133]:
personsdf[personsdf["full_name"]=="Kausik K. Ray"]

,event_id,full_name,first_name,last_name,second_name,country,title,fullname,id
266,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Kausik K. Ray,Kausik,Ray,K.,United Kingdom,,NaN,53c3124a-f56a-4ded-97e4-5e70c661fac5


In [134]:
personsdf[personsdf["full_name"]=="Alberico L. Catapano"]

,event_id,full_name,first_name,last_name,second_name,country,title,fullname,id
231,a12c53cf-a1d6-4def-9c54-a765ccdc3de8,Alberico L. Catapano,Alberico,Catapano,L.,Italy,,NaN,3a785e03-6182-45bd-9c70-cf77ff14208d


In [135]:
personssavedf = personsdf.drop(columns=["full_name"])

In [136]:
with jsonlines.open('../output/structured/Person.jsonl', mode='w') as writer:
    writer.write_all(personssavedf.T.apply(lambda x: x.dropna().to_dict()).tolist())

## SessionPerson and PresentationPerson

In [137]:
all_sessions_raw["class"].unique()

array(['session', 'session_person', 'presentation', 'presentation_person'],
      dtype=object)

In [138]:
sessionpersonsdf = all_sessions_raw[all_sessions_raw["class"]=="session_person"][[
    "full_name", "role", "content_id"]]

In [139]:
len(sessionpersonsdf)

75

In [140]:
[item for item in sessionpersonsdf.content_id if item not in list(sessionsdf.id)]

[]

In [141]:
sessionpersonsdf.rename(columns={"content_id": "session_id"}, inplace=True)

In [142]:
#all_sessions_raw["session_id"] = all_sessions_raw["session_id"].apply(
#    lambda x: first_duplicated_sessions_id if x in list_duplicated_sessions_id else x)

In [143]:
sessionpersonsdf = sessionpersonsdf.merge(personsdf[["full_name", "id"]], on="full_name").rename(
    columns={"id": "person_id"})

In [144]:
sessionpersonsdf.drop(columns=["full_name"], inplace=True)

In [145]:
sessionpersonsdf.head()

,role,session_id,person_id
0,chairperson,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,d3f946fd-0513-4970-8761-efaa6450e40c
1,chairperson,3daf5a49-1466-4b98-b6d0-1cb5391e5bd1,6b87fb58-3335-4ec7-9e44-07724aa2666b
2,chairperson,cb55033f-259d-4b2a-bfee-0207fd941a49,b68458e1-6ad3-4345-bdb9-ffb1f36f7db6
3,chairperson,8f1c99e7-d6b3-4c14-80fd-70755a882167,b68458e1-6ad3-4345-bdb9-ffb1f36f7db6
4,chairperson,f48c08dc-c8f1-4b20-b70c-46d0010eb4bb,b68458e1-6ad3-4345-bdb9-ffb1f36f7db6


In [146]:
# add chairs
sessionpersonsdf = sessionpersonsdf.append(pd.DataFrame([{
    "role": "chairperson",
    "session_id": "d540c69e-b4ed-42dd-ab4d-77726339a8f1",
    "person_id": "350ae66f-7e86-43f4-b870-8f6d264e3fe5"
}]))

In [147]:
# add chairs
sessionpersonsdf = sessionpersonsdf.append(pd.DataFrame([{
    "role": "chairperson",
    "session_id": "af35fc49-2ed2-41c0-a8d2-2681d3b65f7d",
    "person_id": "fa2223a3-80e1-45d8-a9a9-367bb4a891f8"
}]))

In [148]:
sessionpersonsdf[-2:]

,role,session_id,person_id
0,chairperson,d540c69e-b4ed-42dd-ab4d-77726339a8f1,350ae66f-7e86-43f4-b870-8f6d264e3fe5
0,chairperson,af35fc49-2ed2-41c0-a8d2-2681d3b65f7d,fa2223a3-80e1-45d8-a9a9-367bb4a891f8


In [149]:
with jsonlines.open('../output/structured/SessionPerson.jsonl', mode='w') as writer:
    writer.write_all(sessionpersonsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

In [150]:
all_sessions_raw["class"].unique()

array(['session', 'session_person', 'presentation', 'presentation_person'],
      dtype=object)

In [151]:
presentationpersonsdf = all_sessions_raw[all_sessions_raw["class"]=="presentation_person"][[
    "full_name", "role", "content_id"]]

In [152]:
presentationpersonsdf.head()

,full_name,role,content_id
4,Giuseppina Caligiuri,presenter,28dd32ad-8624-47ae-b47a-90ecbdcc4c9a
6,Gary Lewis,presenter,7e23090a-d6b3-4631-8b30-66dc644481e8
8,Kari Alitalo,presenter,f890f8c7-be4b-484e-bcef-cd08e2c73680
10,Giuseppina Caligiuri,presenter,afddf7fc-1f52-439a-a466-1d5562792ebb
15,Stefano Romeo,presenter,b9562485-d68b-4086-8610-a6ad50f27b9b


In [153]:
[item for item in presentationpersonsdf.content_id if item not in list(presentationsdf.id)]

[]

In [154]:
presentationpersonsdf.rename(columns={"content_id": "presentation_id"}, inplace=True)

In [155]:
presentationpersonsdf.head()

,full_name,role,presentation_id
4,Giuseppina Caligiuri,presenter,28dd32ad-8624-47ae-b47a-90ecbdcc4c9a
6,Gary Lewis,presenter,7e23090a-d6b3-4631-8b30-66dc644481e8
8,Kari Alitalo,presenter,f890f8c7-be4b-484e-bcef-cd08e2c73680
10,Giuseppina Caligiuri,presenter,afddf7fc-1f52-439a-a466-1d5562792ebb
15,Stefano Romeo,presenter,b9562485-d68b-4086-8610-a6ad50f27b9b


In [156]:
presentationpersonsdf["role"].unique()

array(['presenter'], dtype=object)

In [157]:
presentationpersonsdf[pd.isnull(presentationpersonsdf["presentation_id"])]

,full_name,role,presentation_id


In [158]:
presentationpersonsdf = presentationpersonsdf.merge(personsdf[["full_name", "id"]], on="full_name").rename(
    columns={"id": "person_id"})

In [159]:
presentationpersonsdf.drop(columns=["full_name"], inplace=True)

In [160]:
presentationpersonsdf.head()

,role,presentation_id,person_id
0,presenter,28dd32ad-8624-47ae-b47a-90ecbdcc4c9a,6b87fb58-3335-4ec7-9e44-07724aa2666b
1,presenter,afddf7fc-1f52-439a-a466-1d5562792ebb,6b87fb58-3335-4ec7-9e44-07724aa2666b
2,presenter,42a63f4c-35a8-4309-a69e-65a5397ee63c,6b87fb58-3335-4ec7-9e44-07724aa2666b
3,presenter,7e23090a-d6b3-4631-8b30-66dc644481e8,190ceddd-414c-4db5-b2dd-2b95c265c723
4,presenter,f890f8c7-be4b-484e-bcef-cd08e2c73680,d3f946fd-0513-4970-8761-efaa6450e40c


In [161]:
with jsonlines.open('../output/structured/PresentationPerson.jsonl', mode='w') as writer:
    writer.write_all(presentationpersonsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

In [162]:
eventdf = pd.DataFrame([{
    "airfinity_id": "AE13189", "city": "virtual", "country": "virtual",
    "downloaded_at": "2021-06-11T15:32:08.464063", "end_date": "2021-06-02",
    "id": personsdf["event_id"].unique()[0], "name": "EAS Congress 2021",
    "source_platform": "ctimeetingtech",
    "source_url": "https://cslide.ctimeetingtech.com/eas21/attendee/confcal/session",
    "start_date": "2020-05-30"
}])

In [163]:
with jsonlines.open('../output/structured/Event.jsonl', mode='w') as writer:
    writer.write_all(eventdf.T.apply(lambda x: x.dropna().to_dict()).tolist())